In [15]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
import time

In [16]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [17]:
# Define database and collection
db = client.nasa_db
collection = db.items

In [18]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [19]:
# Print body of html
print(soup.body.prettify())

<body id="news">
 <svg display="none" height="0" width="0">
  <symbol height="30" id="circle_plus" viewbox="0 0 30 30" width="30">
   <g fill-rule="evenodd" transform="translate(1 1)">
    <circle cx="14" cy="14" fill="#fff" fill-opacity=".1" fill-rule="nonzero" r="14" stroke="inherit" stroke-width="1">
    </circle>
    <path class="the_plus" d="m18.856 12.96v1.738h-4.004v3.938h-1.848v-3.938h-4.004v-1.738h4.004v-3.96h1.848v3.96z" fill="inherit" stroke-width="0">
    </path>
   </g>
  </symbol>
  <symbol height="30" id="circle_arrow" viewbox="0 0 30 30" width="30" xmlns="http://www.w3.org/2000/svg">
   <g transform="translate(1 1)">
    <circle cx="14" cy="14" fill="#fff" fill-opacity=".1" r="14" stroke="inherit" stroke-width="1">
    </circle>
    <path class="the_arrow" d="m8.5 15.00025h7.984l-2.342 2.42c-.189.197-.189.518 0 .715l.684.717c.188.197.494.197.684 0l4.35-4.506c.188-.199.188-.52 0-.717l-4.322-4.48c-.189-.199-.496-.199-.684 0l-.684.716c-.189.197-.189.519 0 .716l2.341 2.419h

In [20]:
# Find article titles
article_titles = soup.find_all('div', class_='content_title')
article_titles

[<div class="content_title">
 <a href="/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/">
 NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/">
 The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8692/nasa-to-hold-mars-2020-perseverance-rover-launch-briefing/">
 NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
 Alabama High School Student Names NASA's Mars Helicopter
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/8645/mars-helicopter-attached-to-nasas-perseverance-rover/">
 Mars Helicopter Attached to NASA's Perseverance Rover
 </a>
 </div>,
 <div class="content_title">
 <a href="/news/864

In [21]:
# Loop to get article titles
for article in article_titles:
    title = article.find('a')
    title_text = title.text
    print(title_text)


NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities


The Launch Is Approaching for NASA's Next Mars Rover, Perseverance


NASA to Hold Mars 2020 Perseverance Rover Launch Briefing


Alabama High School Student Names NASA's Mars Helicopter


Mars Helicopter Attached to NASA's Perseverance Rover


NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes



In [22]:
# Find paragraph text
paragraphs = soup.find_all('div', class_='rollover_description')
paragraphs

[<div class="rollover_description">
 <div class="rollover_description_inner">
 Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.
 </div>
 <div class="overlay_arrow">
 <img alt="More" src="/assets/overlay-arrow.png"/>
 </div>
 </div>,
 <div class="rollover_description">
 <div class="rollover_description_inner">
 The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 
 </div>
 <div class="overlay_arrow">
 <img alt="More" src="/assets/overlay-arrow.png"/>
 </div>
 </div>,
 <div class="rollover_description">
 <div class="rollover_description_inner">
 Learn more about the agency's next Red Planet mission during a live event on June 17.
 </div>
 <div class="overlay_arrow">
 <img alt="More" src="/assets/overlay-arrow.png"/>
 </div>
 </div>,
 <div class="rollover_description">
 <div class="rollo

In [23]:
# Loop through paragraph texts
for paragraph in paragraphs:
    p_text = paragraph.find('div')
    news_p = p_text.text
    print(news_p)


Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.


The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 


Learn more about the agency's next Red Planet mission during a live event on June 17.


Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.


The team also fueled the rover's sky crane to get ready for this summer's history-making launch.


After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.



In [24]:
#JPL Mars Space Images - Featured Image
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [25]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [36]:
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(images_url)
browser.find_by_id("full_image").click()
time.sleep(2)
browser.find_link_by_partial_text("more info").click()
# html = browser.html
soup = BeautifulSoup(browser.html, 'html.parser')
results = soup.find('figure', class_='lede').a.img["src"]

results

featured_img = 'https://www.jpl.nasa.gov'+results

featured_img

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16192_hires.jpg'

In [33]:
# # Find image link with BeautifulSoup
# images = soup.find_all('div', class_='carousel_items')
# images

In [16]:
# Mars Facts. Look at 2/9 Ins Pandas Scraping
url = 'https://space-facts.com/mars/'

In [17]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [18]:
facts = tables[0].rename(columns={0: '', 1: ' '}).set_index('')
facts

,
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
# Use Pandas to convert to HTML table string
facts_html = facts.to_html(index=False)
facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [27]:
#Mars Hemisphere 
#Obtain high resolution images for each hemisphere 
#Use dictionary to store data using th ekeys img_url and title
#Append dictionary with the image url string and hemisphere title to a list.

In [22]:

# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')

In [44]:
#URL reference
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Empty List of hemispheres images url
images_url = []

#Open webpage
browser.visit(url)

# For each hemisphere, pull image url and store with associated hemisphere name
for hemi in hemispheres: 
        hemisphere = {}
        # Click on hemisphere link
        browser.links.find_by_partial_text(hemi).click()
        time.sleep(1)
        # Click on 'Open' to get to full image stored on page
        browser.links.find_by_partial_text('Open').click()
        
        # Pull html
        hemi_html = browser.html
        soup = BeautifulSoup(hemi_html, 'html.parser')
        
        # Pull full-sized image url
        image_url = soup.find('img', class_='wide-image')['src']
        
        # Go back to main page
        browser.back()
        
        # Store image link in dictionary and append to list
#         full_image_url = 'https://astrogeology.usgs.gov' + image_url
        hemisphere['title'] = title
        hemisphere['img_url'] = img
        hemisphere_image_urls.append(hemisphere)
    
        browser.back()
    
images_url
output['images_url'] = images_url

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=62896): Max retries exceeded with url: /session/8dfb93c6bd95774208e49b6bd955825f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb6a1b63eb8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [43]:
print(images_url)

[]
